# Baseline Algorithms with Autoencoder


In [ ]:
# !pip install tensorflow

In [ ]:
# !pip install Keras

In [ ]:
# !pip install h5py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from nltk.tokenize import word_tokenize
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.layers import Layer
import gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
#Preprocessing function
def preprocessing(data_frame):
    ## Preprocessing
    # Removing URLs whithin the tweets
    data_frame["Text"] = data_frame["Text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    # Removing emails, hashtags and punctuations
    data_frame['Text'] = data_frame["Text"].str.replace(r'\S*@\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'#\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'[^\w\s]+', ' ').str.strip()

    # Lowercase Text
    data_frame['Text'] = data_frame['Text'].str.lower()

    # # Removing stopwords
    stop = stopwords.words('english')
    data_frame['Text'].apply(lambda x: [item for item in str(x) if item not in stop])

    # Removing newline characters
    data_frame['Text'] = data_frame['Text'].str.rstrip()

    # Tokenizing Posts and counting the length of each post
    data_frame['Tokens'] = data_frame.apply(lambda row: word_tokenize(str(row['Text'])), axis=1)
    data_frame['Length'] = data_frame.apply(lambda row: len(row['Tokens']), axis=1)

    return data_frame

### Twitter 1

In [ ]:
## Preparing the data
# ## Twitter 10000
Twitter_path = "/content/drive/MyDrive/MH ML project/Datasets/Twitter/twitter-suicidal_data_10000.csv"
df = pd.read_csv(Twitter_path, encoding='latin-1')
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})
df = preprocessing(df)
df

### Reddit S_NS

In [ ]:
## Reddit suicide-not_suicide
Reddit_path = "/content/drive/MyDrive/MH ML project/Datasets/Reddit_non suicide  suicide/Suicide_Detection.csv"
df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'], nrows = 20000)
df_r = df_r.rename(columns={'text': 'Text', 'class': 'Label'})

df = preprocessing(df_r)
label_dict = {'suicide': 1, 'non-suicide': 0}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df['Label']
df

### Reddit Suicide Depression

In [ ]:
## Preparing the data
reddit_path = "/content/drive/MyDrive/MH ML project/Datasets/Reddit_depressionSuicide/reddit_depression_suicidewatch.csv" 

df = pd.read_csv(reddit_path, encoding='latin-1')
df = df.rename(columns={'text':'Text', 'label':'Label'}) 
label_dict = {'depression': 0, 'SuicideWatch': 1}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df = preprocessing(df)
df

Text  Label  \
0      i recently went through a breakup and she said...      0   
1      i do not know how to navigate these feelings  ...      0   
2      so i have been with my bf for 5 months   and h...      0   
3      i am so exhausted of this  just when i think i...      1   
4      i have been severly bullied since i was 5 till...      0   
...                                                  ...    ...   
20358  i took 50mg of seroquel a few hours after i dr...      1   
20359  that is what has happened to me last week  and...      0   
20360  ever just feel alone in a house full of people...      0   
20361  politicians  neighbors  corporations  society ...      0   
20362  i feel like i am just existing  but for what  ...      0   

                                                  Tokens  Length  
0      [i, recently, went, through, a, breakup, and, ...     121  
1      [i, do, not, know, how, to, navigate, these, f...     308  
2      [so, i, have, been, with, my, bf, for, 5, mont...     131  
3      [i, am, so, exhausted, of, this, just, when, i...     349  
4      [i, have, been, severly, bullied, since, i, wa...     198  
...                                                  ...     ...  
20358  [i, took, 50mg, of, seroquel, a, few, hours, a...      35  
20359  [that, is, what, has, happened, to, me, last, ...     119  
20360  [ever, just, feel, alone, in, a, house, full, ...      20  
20361  [politicians, neighbors, corporations, society...      31  
20362  [i, feel, like, i, am, just, existing, but, fo...     133  

[20363 rows x 4 columns]

### Suicide tendency

In [ ]:
## Preparing the data

twitter_path = "/content/drive/MyDrive/MH ML project/Datasets/suicidal-tendency-tweets.csv"  ## positive samples

## reading the abovementioned datasets
df = pd.read_csv(twitter_path, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})

df = preprocessing(df)
df

In [ ]:
ones = list(df['Label']).count(1)
zeros = list(df['Label']).count(0)
difference = zeros-ones
print(ones, zeros, difference, len(df))

In [ ]:
more_ones_path = "/content/drive/MyDrive/MH ML project/Datasets/Twitter/suicideTweetData_56000.csv"
more_ones = pd.read_csv(more_ones_path, encoding='latin-1', nrows = difference)
more_ones = more_ones.rename(columns={'Tweet': 'Text', 'label': 'Label'})
more_ones['Label'] = more_ones['Label'].apply(lambda x: 1)

more_ones
more_ones = preprocessing(more_ones)

In [ ]:
more_ones

In [ ]:
df_1 = df
df_1 = df_1.append(more_ones)
df = df_1

In [ ]:
ones = list(df['Label']).count(1)
zeros = list(df['Label']).count(0)
difference = ones-zeros
print(ones, zeros, difference, len(df))

# Word2Vec

In [ ]:
# model_path = "/content/drive/MyDrive/MH ML project/w2v_models/reddit_SNS/w2v_models_suicide_tendency"
# word2vec embedding
wd2vc_model = gensim.models.Word2Vec(df['Tokens'], min_count = 1, size = 300, window = 5)
wd2vc_model.train(df['Tokens'],epochs=10,total_examples=len(df['Tokens']))
# wd2vc_model.save(model_path)

(5837860, 7617330)

In [ ]:
# words most similar to a given word.
similar_words = wd2vc_model.wv.most_similar('kill')
print(similar_words)


In [ ]:
## Creating the embedding matrix

vocab = wd2vc_model.wv.vocab
print("The total number of words are : ", len(vocab))
vocab = list(vocab.keys())

word_vec_dict = {}
for word in vocab:
    word_vec_dict[word] = wd2vc_model.wv.get_vector(word)
print("The no of key-value pairs : ", len(word_vec_dict))

max_length = max(df['Length'])
print("maximum length = ", max_length)

tok = Tokenizer()
tok.fit_on_texts(df['Tokens'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['Tokens'])

embed_dim = 300
pad_rev = pad_sequences(encd_rev, maxlen=max_length, padding='post')
print("pad_shape = ", pad_rev.shape)


embed_matrix = np.zeros(shape=(vocab_size, embed_dim))
print("embedding matrix shape = ", embed_matrix.shape)
for word, i in tok.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None: 
        embed_matrix[i] = embed_vector

In [ ]:
pad_rev

In [ ]:
np.count_nonzero(np.isnan(pad_rev))

In [ ]:
## Spliting the data
X, X_test, y, y_test = train_test_split(pad_rev, df['Label'], shuffle=True, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=33)

print("Train = ", X_train.shape, y_train.shape)
print("Test = ", X_test.shape, y_test.shape)
print("Validation = ",X_val.shape, y_val.shape)


# Autoencoder

### Simple AE

In [ ]:
arrAEFeatures = []

def build_AE(X_train):
    input_data = Input(shape=(X_train.shape[1],))
    
    encoded = Sequential()
    #Encoder
    encoded = Dense(256, activation='relu',name="encode_layer1")(input_data)
    encoded = Dense(128, activation='relu',name="encode_layer2")(encoded)
    encoded = Dense(64, activation='relu',name="encode_layer3")(encoded)

    #Decoder
    decoded = Dense(128, activation='relu',name="decode_layer1")(encoded)
    decoded = Dense(256, activation='relu',name="decode_layer2")(decoded)
    decoded = Dense(X_train.shape[1], activation='sigmoid',name="decode_layer3")(decoded)

    autoencoder = Model(input_data, decoded)
    autoencoder.compile(optimizer='adadelta', loss='mse')
    autoencoder.summary()
    return autoencoder


In [ ]:
autoencoder = build_AE(X_train)
autoencoder.fit(X_train, X_train,epochs=3,batch_size=256,validation_data=(X_test, X_test))

In [ ]:
encoder = Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('encode_layer3').output)

### LSTM AE

In [ ]:
# import tensorflow as tf
# from keras.backend import set_session
# config = tf.compat.v1.ConfigProto()

# config.gpu_options.per_process_gpu_memory_fraction = 0.7
# config.gpu_options.allow_growth = True 
# set_session(tf.compat.v1.Session(config=config))

In [ ]:
n_inputs = X_train.shape[1]
print(X_train.shape, n_inputs)

In [ ]:
from keras.layers import *

model = Sequential()
model.add(Input(shape=(n_inputs,1)))
model.add(LSTM(128, input_shape = (n_inputs,1) , return_sequences=True))
model.add(LeakyReLU(alpha=0.05))
model.add(LSTM(64, return_sequences=False))
model.add(LeakyReLU(alpha=0.05))
model.add(RepeatVector(n_inputs))
model.add(LSTM(64, return_sequences=True))
model.add(LeakyReLU(alpha=0.05))
model.add(LSTM(128,return_sequences=True))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(1))

model.summary()

In [ ]:
from tensorflow.keras.optimizers import *
opt = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=True, clipnorm=0.5, clipvalue = 0.5)

model.compile(loss = 'mse', 
               optimizer = opt)

history = model.fit(X_train, X_train, epochs=3, batch_size=32, verbose=True, validation_data=(X_test,X_test))

In [ ]:
encoder = Model(inputs=model.inputs, outputs=model.layers[2].output)


In [ ]:
# plot the autoencoder

from tensorflow.keras.utils import plot_model
plot_model(encoder, 'LSTM_autoencoder.png', show_shapes=True)

In [ ]:
# encoder.save('/content/drive/MyDrive/MH ML project/Encoders/RSNS_LSTM_AE_model.h5')

In [ ]:
# from keras.models import load_model
# encoder = load_model('/content/drive/MyDrive/MH ML project/Encoders/RSNS_LSTM_AE_model.h5')

### Contractive AE

In [ ]:
import tensorflow as tf
    
tf.compat.v1.disable_eager_execution()

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
import keras.backend as K

N_batch = 32
N_epochs = 10

lam = 1e-4
inputs = Input(shape=(X_train.shape[1],))
encoded = Dense(64, activation='sigmoid', name='encoded')(inputs)
outputs = Dense(N, activation='linear')(encoded)

model = Model(inputs, outputs)

def contractive_loss(y_pred, y_true):
    mse = K.mean(K.square(y_true - y_pred), axis=1)

    W = K.variable(value=model.get_layer('encoded').get_weights()[0])
    W = K.transpose(W)
    h = model.get_layer('encoded').output
    dh = h * (1 - h)

    contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

    return mse + contractive

In [ ]:
model.compile(optimizer='adam', loss=contractive_loss)
model.fit(X_train, X_train, batch_size=N_batch, epochs=N_epochs, validation_data=(X_val, X_val))

In [ ]:
encoder = Model(inputs=inputs, outputs=encoded)

In [ ]:
# plot the autoencoder
from tensorflow.keras.utils import plot_model
plot_model(encoder, 'contractive_autoencoder.png', show_shapes=True)

### Sparse AE

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.layers.convolutional import Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l1

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

def sparse_autoencoder(X, lam=1e-5):
    X = X.reshape(X.shape[0], -1)
    M, N = X.shape

    inputs = Input(shape=(N,))
    h = Dense(64, activation='sigmoid', activity_regularizer=l1(lam))(inputs)
    outputs = Dense(N)(h)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, X, batch_size=64, epochs=10, validation_data=(X_val, X_val))

    return Model(inputs, h)

In [ ]:
encoder = sparse_autoencoder(X_train)

In [ ]:
# plot the autoencoder

from tensorflow.keras.utils import plot_model
plot_model(encoder, 'sparse_autoencoder.png', show_shapes=True)

### Convolutional AE

In [ ]:
X_train_reshaped = X_train.reshape(-1, 1, X_train.shape[1])
X_test_reshaped  = X_test.reshape(-1, 1, X_test.shape[1])
X_val_reshaped = X_val.reshape(-1, 1, X_val.shape[1])

In [ ]:
print("Train = ", X_train_reshaped.shape, y_train.shape)
print("Test = ", X_test_reshaped.shape, y_test.shape)
print("Validation = ",X_val_reshaped.shape, y_val.shape)

In [ ]:
from keras.layers import Conv1D, Conv1DTranspose
from keras.layers import Input
from keras.layers import MaxPooling1D, UpSampling1D
from keras.models import Model

n_inputs = X_train_reshaped.shape[2]
print(X_train_reshaped.shape, n_inputs)

input_data = Input(shape=(None, n_inputs))

enc_conv1 = Conv1D(128, (3, ), activation='relu', padding='same')(input_data)
enc_pool1 = MaxPooling1D(4, padding='same')(enc_conv1)
enc_conv2 = Conv1D(64, (4, ), activation='relu', padding='same')(enc_pool1)
enc_ouput = MaxPooling1D(2, padding='same')(enc_conv2)

dec_conv2 = Conv1DTranspose(64, (4, ), activation='relu', padding='same')(enc_ouput)
dec_conv3 = Conv1DTranspose(128, (3, ), activation='relu')(dec_conv2)
dec_output = Conv1D(n_inputs, (1,), activation="sigmoid", padding="same")(dec_conv3)

cnn_autoencoder = Model(input_data, dec_output)
cnn_autoencoder.compile(optimizer='rmsprop', loss='mse') 
 
cnn_autoencoder.summary()

In [ ]:
cnn_autoencoder.fit(X_train_reshaped, X_train_reshaped, epochs=5, batch_size=16, verbose=True, validation_data=(X_test_reshaped,X_test_reshaped))

In [ ]:
encoder = Model(input_data, enc_ouput)

In [ ]:
# plot the autoencoder
from tensorflow.keras.utils import plot_model
plot_model(encoder, 'cnn_autoencoder.png', show_shapes=True)

# Applying the AE

In [ ]:
# X_train_encoded = encoder.predict(X_train_reshaped)
# X_val_encoded = encoder.predict(X_val_reshaped)
# X_test_encoded = encoder.predict(X_test_reshaped)

# print("Train = ", X_train_encoded_reshaped.shape, y_train_reshaped.shape)
# print("Test = ", X_test_encoded_reshaped.shape, y_test_reshaped.shape)
# print("Validation = ", X_val_encoded_reshaped.shape, y_val_reshaped.shape)

In [ ]:
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)
X_test_encoded = encoder.predict(X_test)

In [ ]:
print("Train = ", X_train_encoded.shape, y_train.shape)
print("Test = ", X_test_encoded.shape, y_test.shape)
print("Validation = ", X_val_encoded.shape, y_val.shape)

In [ ]:
X_train_encoded_reshaped = X_train_encoded.reshape(-1, 1, X_train_encoded.shape[1])
X_test_encoded_reshaped  = X_test_encoded.reshape(-1, 1, X_test_encoded.shape[1])
X_val_encoded_reshaped = X_val_encoded.reshape(-1, 1, X_val_encoded.shape[1])

In [ ]:
y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)
y_val_reshaped = y_val.values.reshape(-1, 1)

In [ ]:
print("Train = ", X_train_encoded.shape, y_train_reshaped.shape)
print("Test = ", X_test_encoded.shape, y_test_reshaped.shape)
print("Validation = ", X_val_encoded.shape, y_val_reshaped.shape)

### Saving and loading encoded vectors

In [ ]:
# # save numpy array as csv file
# from numpy import asarray
# from numpy import savetxt
# savetxt('/content/drive/MyDrive/MH ML project/Algorithm/encoded_arrays/X_train_encoded.csv', X_train_encoded, delimiter=',')

In [ ]:
# # save numpy array as csv file
# from numpy import asarray
# from numpy import savetxt
# savetxt('/content/drive/MyDrive/MH ML project/Algorithm/encoded_arrays/X_val_encoded.csv', X_val_encoded, delimiter=',')

In [ ]:
# # save numpy array as csv file
# from numpy import asarray
# from numpy import savetxt
# savetxt('/content/drive/MyDrive/MH ML project/Algorithm/encoded_arrays/X_test_encoded.csv', X_test_encoded, delimiter=',')

In [ ]:
# from numpy import loadtxt
# # load array
# X_train_encoded = loadtxt('/content/drive/MyDrive/MH ML project/Algorithm/encoded_arrays/X_train_encoded.csv', delimiter=',')
# X_test_encoded = loadtxt('/content/drive/MyDrive/MH ML project/Algorithm/encoded_arrays/X_test_encoded.csv', delimiter=',')
# X_val_encoded = loadtxt('/content/drive/MyDrive/MH ML project/Algorithm/encoded_arrays/X_val_encoded.csv', delimiter=',')
# # print the array
# print(X_train_encoded.shape)
# print(X_test_encoded.shape)
# print(X_val_encoded.shape)

In [ ]:
#  ## mode of padding is constant == zero
# X_train_padded = np.pad(X_train_encoded, pad_width = ((0,0),(0, embed_dim-64)))
# X_test_padded = np.pad(X_test_encoded, pad_width = ((0,0),(0, embed_dim-64)))
# X_val_padded = np.pad(X_val_encoded, pad_width = ((0,0),(0, embed_dim-64)))

# print("Train = ", X_train_padded.shape, y_train.shape)
# print("Test = ", X_test_padded.shape, y_test.shape)
# print("Validation = ", X_val_padded.shape, y_val.shape)

# Baselines

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn import metrics
def evaluate_classfier_quality(pred,y_test):
    accuracy = metrics.accuracy_score(y_test,pred)
    print("Accuracy: %1.4f" %accuracy)
    precision = metrics.precision_score(y_test,pred,average='weighted')
    print("Precision: %1.4f" % precision)
    recall = metrics.recall_score(y_test,pred,average='weighted')
    print("Recall: %1.4f" % recall)
    f1 = metrics.f1_score(y_test,pred,average='weighted')
    print("F1: %1.4f" % f1)
    return accuracy,precision,recall,f1

### ReCreating 2D shape data

In [ ]:
X_train_new = np.zeros(shape=(X_train_encoded.shape[0], X_train_encoded.shape[2]))

for row in range(X_train_new.shape[0]):
  X_train_new[row]=X_train_encoded[row][0]

print(X_train_new)

In [ ]:
X_test_new = np.zeros(shape=(X_test_encoded.shape[0], X_test_encoded.shape[2]))

for row in range(X_test_new.shape[0]):
  X_test_new[row]=X_test_encoded[row][0]

print(X_test_new)

In [ ]:
X_train_encoded = X_train_new
X_test_encoded = X_test_new

### Kmeans

In [ ]:
from sklearn.cluster import KMeans
sse = []
num_of_k = list(range(2, 6))

kmeans = KMeans(n_clusters=2)
results = kmeans.fit(X_train_encoded)
y_pred = kmeans.predict(X_test_encoded)

In [ ]:
km = evaluate_classfier_quality(y_pred,y_test)
km

### Random Forest

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_encoded, y_train)
y_pred = model_rf.predict(X_test_encoded)

In [ ]:
RF = evaluate_classfier_quality(y_pred,y_test)
RF

### Decision Tree

In [ ]:
grid_param={'min_samples_split' : [10,50,100],'max_depth': [2,4,8]}
model_dt = GridSearchCV(DecisionTreeClassifier(),param_grid=grid_param,cv=5,n_jobs=-1)
model_dt.fit(X_train_encoded,y_train)
y_pred = model_dt.predict(X_test_encoded)
evaluate_classfier_quality(y_pred,y_test)

### Gradient Boost

In [ ]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train_encoded,y_train)
y_pred = model_gb.predict(X_test_encoded)
evaluate_classfier_quality(y_pred,y_test)

### SVM

In [ ]:
from sklearn.svm import SVC
model_svm = SVC(kernel='linear')
model_svm.fit(X_train, y_train)
y_pred = model_svm.predict(X_test)
evaluate_classfier_quality(y_pred,y_test)

### Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train_encoded, y_train)
y_pred = logreg.predict(X_test_encoded)
evaluate_classfier_quality(y_pred,y_test)

### LSTM-CNN

In [ ]:
# X_train_encoded_reshaped = X_train_encoded
# X_test_encoded_reshaped  = X_test_encoded
# X_val_encoded_reshaped = X_val_encoded

In [ ]:
X_train_encoded_reshaped = X_train_encoded.reshape(-1, 1, X_train_encoded.shape[1])
X_test_encoded_reshaped  = X_test_encoded.reshape(-1, 1, X_test_encoded.shape[1])
X_val_encoded_reshaped = X_val_encoded.reshape(-1, 1, X_val_encoded.shape[1])

In [ ]:
y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)
y_val_reshaped = y_val.values.reshape(-1, 1)

In [ ]:
print("Train = ", X_train_encoded_reshaped.shape, y_train_reshaped.shape)
print("Test = ", X_test_encoded_reshaped.shape, y_test_reshaped.shape)
print("Validation = ", X_val_encoded_reshaped.shape, y_val_reshaped.shape)

In [ ]:
from keras.layers.activation.leaky_relu import LeakyReLU

lstm_cnn_model = tf.keras.models.Sequential()
lstm_cnn_model.add(tf.keras.layers.Input(shape=(1, X_train_encoded_reshaped.shape[2])))
lstm_cnn_model.add(tf.keras.layers.Dropout(0.5))
lstm_cnn_model.add(tf.keras.layers.LSTM(units=100, input_shape=(1,64), return_sequences = True))
lstm_cnn_model.add(tf.keras.layers.Conv1D(3, (8,), padding='same', activation='relu'))
lstm_cnn_model.add(tf.keras.layers.Flatten())
lstm_cnn_model.add(Dense(1, activation='sigmoid')) ##4labels

lstm_cnn_model.summary()

In [ ]:
## Compiling and fitting
lstm_cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = lstm_cnn_model.fit(X_train_encoded_reshaped, y_train_reshaped, 
              epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val_encoded_reshaped, y_val_reshaped))

In [ ]:
#Prediction on the test data
pred = lstm_cnn_model.predict(X_test_encoded_reshaped)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
    # y_pred[item] = list(pred[item]).index(max(pred[item]))
    if pred[item] >= 0.5:
      y_pred[item] = 1
    else:
      y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)

### CNN

In [ ]:
from keras.layers.activation.leaky_relu import LeakyReLU
from keras.initializers import Constant

cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Input(shape=(1, X_train_encoded_reshaped.shape[2])))
cnn_model.add(tf.keras.layers.Dropout(0.5))
cnn_model.add(tf.keras.layers.Conv1D(3, (8,), padding='same'))
cnn_model.add(tf.keras.layers.LeakyReLU(0.05))

cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(Dense(1, activation='sigmoid')) ##4labels

cnn_model.summary()

In [ ]:
## Compiling and fitting
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = cnn_model.fit(X_train_encoded_reshaped, y_train_reshaped, 
              epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val_encoded_reshaped, y_val_reshaped))

In [ ]:
#Prediction on the test data
pred = cnn_model.predict(X_test_encoded_reshaped)
print(pred)

In [ ]:
len(y_test_reshaped)
y_test = y_test_reshaped

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
    # y_pred[item] = list(pred[item]).index(max(pred[item]))
    if pred[item] >= 0.5:
      y_pred[item] = 1
    else:
      y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)

### LSTM

In [ ]:
from keras.layers.activation.leaky_relu import LeakyReLU
from keras.initializers import Constant

lstm_model = tf.keras.models.Sequential()
lstm_model.add(tf.keras.layers.Input(shape=(1, X_train_encoded_reshaped.shape[2])))
lstm_model.add(tf.keras.layers.Dropout(0.5))
lstm_model.add(tf.keras.layers.LSTM(units=100, input_shape=(1,64), return_sequences = True))
lstm_model.add(tf.keras.layers.Flatten())
lstm_model.add(Dense(1, activation='sigmoid')) ##4labels

lstm_model.summary()

In [ ]:
## Compiling and fitting
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = lstm_model.fit(X_train_encoded_reshaped, y_train_reshaped, 
              epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val_encoded_reshaped, y_val_reshaped))

In [ ]:
#Prediction on the test data
pred = lstm_model.predict(X_test_encoded_reshaped)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
    # y_pred[item] = list(pred[item]).index(max(pred[item]))
    if pred[item] >= 0.5:
      y_pred[item] = 1
    else:
      y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)